#### TODO:
1. Work on Assembly in "# get activations" part instead of the StimulusSet
2. Translate to ``behavior.py`` and the test.

In [78]:
import functools

import numpy as np

import brainscore_vision
from brainio.assemblies import DataAssembly, BehavioralAssembly, walk_coords
from brainscore_vision.benchmark_helpers.screen import place_on_screen
from brainscore_vision.model_helpers.activations import PytorchWrapper
from brainscore_vision.model_helpers.brain_transformation import ModelCommitment

In [79]:
def pytorch_custom():
    import torch
    from torch import nn
    from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images

    class MyModel(nn.Module):
        def __init__(self):
            super(MyModel, self).__init__()
            np.random.seed(0)
            torch.random.manual_seed(0)
            self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)
            self.relu1 = torch.nn.ReLU()
            linear_input_size = np.power((224 - 3 + 2 * 0) / 1 + 1, 2) * 2
            self.linear = torch.nn.Linear(int(linear_input_size), 1000)
            self.relu2 = torch.nn.ReLU()

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = x.view(x.size(0), -1)
            x = self.linear(x)
            x = self.relu2(x)
            return x

    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    return PytorchWrapper(model=MyModel(), preprocessing=preprocessing)

In [80]:
# create model
activations_model = pytorch_custom()
layers = ["relu2"]

# create brain model
brain_model = ModelCommitment(
    identifier=activations_model.identifier, 
    activations_model=activations_model, 
    layers=[None], 
    behavioral_readout_layer='relu2')

# get activations
assy = brainscore_vision.load_dataset(f'Hebart2023')
stimuli = place_on_screen(
    stimulus_set=assy.stimulus_set,
    target_visual_degrees=brain_model.visual_degrees(),
    source_visual_degrees=8)

In [81]:
def calculate_similarity_matrix(features, similarity_measure='dot'):
   print(features, type(features))
   features = features.transpose('presentation', 'neuroid')
   values = features.values
   if similarity_measure == 'dot':
      similarity_matrix = np.dot(values, np.transpose(values))
   elif similarity_measure == 'cosine':
      row_norms = np.linalg.norm(values, axis=1).reshape(-1, 1)
      norm_product = np.dot(row_norms, row_norms.T)
      dot_product = np.dot(values, np.transpose(values))
      similarity_matrix = dot_product / norm_product
   else:
      raise ValueError(
      f"Unknown similarity_measure {similarity_measure} -- expected one of 'dot' or 'cosine'")

   similarity_matrix = DataAssembly(similarity_matrix, coords={
        **{f"{coord}_left": ('presentation_left', values) for coord, _, values in
           walk_coords(features['presentation'])},
        **{f"{coord}_right": ('presentation_right', values) for coord, _, values in
           walk_coords(features['presentation'])}
   }, dims=['presentation_left', 'presentation_right'])
   return similarity_matrix

In [82]:
def calculate_choices(similarity_matrix, triplets):
    triplets = np.array(triplets).reshape(-1, 3)
    choice_predictions = []
    for triplet in triplets:
        i, j, k = triplet
        sims = similarity_matrix[i, j], similarity_matrix[i, k],  similarity_matrix[j, k]
        idx = triplet[2 - np.argmax(sims)]
        choice_predictions.append(idx)
    # TODO return as DataAssembly
    return choice_predictions

In [83]:
# TODO: This should become look_at(), Make sure to drop duplicates where necessary!
features = activations_model(stimuli, layers=layers)
features = features.transpose('presentation', 'neuroid')
assy = brainscore_vision.load_dataset(f'Hebart2023')
triplets = np.array([assy['image_1'], assy['image_2'], assy['image_3']]).T
triplets = triplets.reshape(-1)

In [84]:
sample = triplets[:10*3]
sim = calculate_similarity_matrix(features, similarity_measure='cosine')
choices = calculate_choices(similarity_matrix=sim, triplets=sample)

<xarray.NeuroidAssembly (presentation: 1854, neuroid: 1000)>
array([[0.01105666, 0.05785533, 0.        , ..., 0.1394486 , 0.        ,
        0.03779155],
       [0.        , 0.        , 0.        , ..., 0.01137817, 0.        ,
        0.13863228],
       [0.3239666 , 0.30983254, 0.        , ..., 0.2911486 , 0.        ,
        0.1254089 ],
       ...,
       [0.01853104, 0.22858877, 0.        , ..., 0.04627881, 0.14518598,
        0.        ],
       [0.1675643 , 0.03827018, 0.        , ..., 0.20744947, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.22435561, 0.02024   ,
        0.34257683]], dtype=float32)
Coordinates:
  * neuroid           (neuroid) MultiIndex
  - neuroid_num       (neuroid) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  - model             (neuroid) object 'MyModel' 'MyModel' ... 'MyModel'
  - layer             (neuroid) object 'relu2' 'relu2' ... 'relu2' 'relu2'
  - channel           (neuroid) int64 0 1 2 3 4 5 6 ... 994 995

In [85]:
#TODO: this needs to be fixed
#stimulus_ids = assy['stimulus_id'][:10]
#choices = BehavioralAssembly(choices, coords={
#    'triplet_index': ('presentation', [i for i in range(0, len(stimulus_ids), 3)]),
#    'triplet_stimulus_id0': ('presentation', [{stimulus_ids[i]} for i in range(0, len(stimulus_ids), 3)]),
#    'triplet_stimulus_id1': ('presentation', [{stimulus_ids[i+1]} for i in range(0, len(stimulus_ids), 3)]),
#    'triplet_stimulus_id2': ('presentation', [{stimulus_ids[i+2]} for i in range(0, len(stimulus_ids), 3)]),
#    'stimulus_id': ('presentation', [f"{stimulus_ids[i]}__{stimulus_ids[i+1]}__{stimulus_ids[i+2]}" for i in range(0, len(stimulus_ids), 3)])
#    }, dims=['presentation'])

In [86]:
print(sample)
print(choices)

[ 112 1459  632 1278 1561  792  796 1660  413   27  671  320 1448 1531
  324 1594  280 1825 1381 1522   14 1528  526  208 1491 1336 1850 1494
  171 1113]
[1459, 1278, 1660, 320, 1531, 1825, 14, 1528, 1850, 1113]
